In [1]:
# #use data_team_concurrent iam
# dbutils.fs.cp("s3://ehth-databricks-poc-test/ehealth/JingP/cora/cora.cites","dbfs:/tmp/cora/cora.cites")
# dbutils.fs.cp("s3://ehth-databricks-poc-test/ehealth/JingP/cora/cora.content","dbfs:/tmp/cora/cora.content")

In [2]:
# #then use horovod cluster
# dbutils.fs.cp("dbfs:/tmp/cora/cora.cites","file:/tmp/cora/cora.cites")
# dbutils.fs.cp("dbfs:/tmp/cora/cora.content","file:/tmp/cora/cora.content")

In [3]:
%run ./utils

In [4]:
%run ./layers/graph

In [5]:
import os
import time

checkpoint_dir = '/dbfs/kegra_horovod/train/{}/'.format(time.time())
os.makedirs(checkpoint_dir)

In [6]:
from __future__ import print_function

from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

#databricks import functions in the library by %run
#from kegra.layers.graph import GraphConvolution
#from kegra.utils import *

import time

# Horovod: Import the relevant submodule
import horovod.keras as hvd
from keras import backend as K
import tensorflow as tf
import keras

In [7]:
# Define parameters
DATASET = 'cora'
PATH='/dbfs/tmp/cora/'
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
LEARNING_RATE=0.01

In [8]:
%sh ls /tmp/cora

cora.cites
cora.content

In [9]:
def train_hvd(learning_rate=LEARNING_RATE):
  # Horovod: initialize Horovod.
  hvd.init()
  
  # Get data
  X, A, y = load_data(path=PATH,dataset=DATASET)
  y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

  # Normalize X
  X /= X.sum(1).reshape(-1, 1)
  
  if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
    print('Using local pooling filters...')
    A_ = preprocess_adj(A, SYM_NORM)
    support = 1
    graph = [X, A_]
    G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

  elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    print('Using Chebyshev polynomial basis filters...')
    L = normalized_laplacian(A, SYM_NORM)
    L_scaled = rescale_laplacian(L)
    T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)
    support = MAX_DEGREE + 1
    graph = [X]+T_k
    G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True) for _ in range(support)]

  else:
      raise Exception('Invalid filter type.')

  X_in = Input(shape=(X.shape[1],))
  
  print("X_in shape is {}".format(X_in.shape))
   
  # Define model architecture
  # NOTE: We pass arguments for graph convolutional layers as a list of tensors.
  # This is somewhat hacky, more elegant options would require rewriting the Layer base class.
  H = Dropout(0.5)(X_in)
  H = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
  H = Dropout(0.5)(H)
  Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

  # Compile model
  model = Model(inputs=[X_in]+G, outputs=Y)
  model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate))
  
  print ("model compile is done")
  
  callbacks = [
      # Horovod: broadcast initial variable states from rank 0 to all other processes.
      # This is necessary to ensure consistent initialization of all workers when
      # training is started with random weights or restored from a checkpoint.
      hvd.callbacks.BroadcastGlobalVariablesCallback(0),
  ]

  # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
  if hvd.rank() == 0:
      callbacks.append(keras.callbacks.ModelCheckpoint(checkpoint_dir + '/checkpoint-{epoch}.ckpt', save_weights_only = True,period=100))
      
  
  print ("hvd call back done")
  
  

  # Helper variables for main training loop
  wait = 0
  preds = None
  best_val_loss = 99999

  # Fit

  # Log wall-clock time
 
  # Single training iteration (we mask nodes without labels for loss calculation)
  model.fit(graph, y_train, sample_weight=train_mask,\
            batch_size=A.shape[0], epochs=NB_EPOCH, verbose=2, shuffle=False)
        

t = time.time()  
train_hvd()
print ("time cost is {}".format(time.time()-t))
#print(X_in.shape)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.
Using local pooling filters...
X_in shape is (?, 1433)
model compile is done
hvd call back done
Epoch 1/200
 - 1s - loss: 1.9614
Epoch 2/200
 - 0s - loss: 1.9490
Epoch 3/200
 - 0s - loss: 1.9327
Epoch 4/200
 - 0s - loss: 1.9191
Epoch 5/200
 - 0s - loss: 1.9008
Epoch 6/200
 - 0s - loss: 1.8816
Epoch 7/200
 - 0s - loss: 1.8722
Epoch 8/200
 - 0s - loss: 1.8580
Epoch 9/200
 - 0s - loss: 1.8431
Epoch 10/200
 - 0s - loss: 1.8265
Epoch 11/200
 - 0s - loss: 1.8158
Epoch 12/200
 - 0s - loss: 1.7890
Epoch 13/200
 - 0s - loss: 1.7913
Epoch 14/200
 - 0s - loss: 1.7919
Epoch 15/200
 - 0s - loss: 1.7711
Epoch 16/200
 - 0s - loss: 1.7483
Epoch 17/200
 - 0s - loss: 1.7536
Epoch 18/200
 - 0s - loss: 1.7612
Epoch 19/200
 - 0s - loss: 1.7388
Epoch 20/200
 - 0s - loss: 1.7350
Epoch 21/200
 - 0s - loss: 1.7234
Epoch 22/200
 - 0s - loss: 1.7332
Epoch 23/200
 - 0s - loss: 1.7180
Epoch 24/200
 - 0s - loss: 1.7018
Epoch 25/200
 - 0s - loss: 1.7013
Epoch 26/200
 - 0s - loss: 1.7035
Epoch 27/200
 - 0s - loss: 1.6541
Epoch 28/200
 - 0s - loss: 1.6566
Epoch 29/200
 - 0s - loss: 1.6493
Epoch 30/200
 - 0s - loss: 1.6619
Epoch 31/200
 - 0s - loss: 1.6481
Epoch 32/200
 - 0s - loss: 1.6393
Epoch 33/200
 - 0s - loss: 1.6386
Epoch 34/200
 - 0s - loss: 1.6066
Epoch 35/200
 - 0s - loss: 1.6335
Epoch 36/200
 - 0s - loss: 1.5966
Epoch 37/200
 - 0s - loss: 1.5762
Epoch 38/200
 - 0s - loss: 1.5958
Epoch 39/200
 - 0s - loss: 1.5736
Epoch 40/200
 - 0s - loss: 1.5382
Epoch 41/200
 - 0s - loss: 1.5532
Epoch 42/200
 - 0s - loss: 1.5571
Epoch 43/200
 - 0s - loss: 1.5429
Epoch 44/200
 - 0s - loss: 1.5419
Epoch 45/200
 - 0s - loss: 1.5384
Epoch 46/200
 - 0s - loss: 1.5132
Epoch 47/200
 - 0s - loss: 1.4964
Epoch 48/200
 - 0s - loss: 1.5111
Epoch 49/200
 - 0s - loss: 1.5021
Epoch 50/200
 - 0s - loss: 1.4796
Epoch 51/200
 - 0s - loss: 1.4382
Epoch 52/200
 - 0s - loss: 1.4127
Epoch 53/200
 - 0s - loss: 1.4512
Epoch 54/200
 - 0s - loss: 1.4428
Epoch 55/200
 - 0s - loss: 1.4319
Epoch 56/200
 - 0s - loss: 1.4416
Epoch 57/200
 - 0s - loss: 1.4212
Epoch 58/200
 - 0s - loss: 1.4067
Epoch 59/200
 - 0s - loss: 1.3871
Epoch 60/200
 - 0s - loss: 1.4097
Epoch 61/200
 - 0s - loss: 1.3804
Epoch 62/200
 - 0s - loss: 1.3982
Epoch 63/200
 - 0s - loss: 1.3521
Epoch 64/200
 - 0s - loss: 1.3961
Epoch 65/200
 - 0s - loss: 1.3387
Epoch 66/200
 - 0s - loss: 1.3338
Epoch 67/200
 - 0s - loss: 1.3256
Epoch 68/200
 - 0s - loss: 1.3609
Epoch 69/200
 - 0s - loss: 1.3234
Epoch 70/200
 - 0s - loss: 1.2976
Epoch 71/200
 - 0s - loss: 1.3421
Epoch 72/200
 - 0s - loss: 1.3315
Epoch 73/200
 - 0s - loss: 1.3029
Epoch 74/200
 - 0s - loss: 1.2673
Epoch 75/200
 - 0s - loss: 1.2915
Epoch 76/200
 - 0s - loss: 1.2600
Epoch 77/200
 - 0s - loss: 1.2933
Epoch 78/200
 - 0s - loss: 1.2634
Epoch 79/200
 - 0s - loss: 1.2574
Epoch 80/200
 - 0s - loss: 1.2289
Epoch 81/200
 - 0s - loss: 1.2692
Epoch 82/200
 - 0s - loss: 1.2301
Epoch 83/200
 - 0s - loss: 1.2115
Epoch 84/200
 - 0s - loss: 1.2131
Epoch 85/200
 - 0s - loss: 1.1902
Epoch 86/200
 - 0s - loss: 1.2002
Epoch 87/200
 - 0s - loss: 1.2063
Epoch 88/200
 - 0s - loss: 1.1956
Epoch 89/200
 - 0s - loss: 1.1743
Epoch 90/200
 - 0s - loss: 1.2278
Epoch 91/200
 - 0s - loss: 1.1463
Epoch 92/200
 - 0s - loss: 1.2096
Epoch 93/200
 - 0s - loss: 1.1334
Epoch 94/200
 - 0s - loss: 1.1577
Epoch 95/200
 - 0s - loss: 1.1360
Epoch 96/200
 - 0s - loss: 1.1596
Epoch 97/200
 - 0s - loss: 1.1449
Epoch 98/200
 - 0s - loss: 1.0899
Epoch 99/200
 - 0s - loss: 1.1565
Epoch 100/200
 - 0s - loss: 1.1353
Epoch 101/200
 - 0s - loss: 1.0973
Epoch 102/200
 - 0s - loss: 1.1112
Epoch 103/200
 - 0s - loss: 1.0983
Epoch 104/200
 - 0s - loss: 1.1623
Epoch 105/200
 - 0s - loss: 1.1375
Epoch 106/200
 - 0s - loss: 1.0988
Epoch 107/200
 - 0s - loss: 1.1168
Epoch 108/200
 - 0s - loss: 1.1126
Epoch 109/200
 - 0s - loss: 1.1307
Epoch 110/200
 - 0s - loss: 1.0392
Epoch 111/200
 - 0s - loss: 1.1252
Epoch 112/200
 - 0s - loss: 1.0721
Epoch 113/200
 - 

In [10]:
from sparkdl import HorovodRunner
t = time.time()
hr = HorovodRunner(np=8)
model = hr.run(train_hvd, learning_rate=LEARNING_RATE)
print ("time cost is {}".format(time.time()-t))

The global names read or written to by the pickled function are {'PATH', 'Adam', 'SYM_NORM', 'keras', 'FILTER', 'print', 'Dropout', 'hvd', 'Input', 'GraphConvolution', 'DATASET', 'normalized_laplacian', 'chebyshev_polynomial', 'get_splits', 'NB_EPOCH', 'Exception', 'Model', 'preprocess_adj', 'rescale_laplacian', 'checkpoint_dir', 'range', 'l2', 'MAX_DEGREE', 'load_data'}.
The pickled object size is 10163 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Start training.
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
[1,0]<stderr>:Using TensorFlow backend.
[1,3]<stdout>:Loading cora dataset...
[1,6]<stdout>:Loading cora dataset...
[1,2]<stdout>:Loading cora dataset...
[1,1]<stdout>:Loading cora dataset...
[1,5]<stdout>:Loading cora dataset...
[1,4]<stdout>:Loading cora dataset...
[1,7]<stdout>:Loading cora dataset...
[1,0]<stdout>:Loading cora dataset...
[1,7]<stdout>:Dataset has 2708 nodes, 5429 edges, 1433 features.
[1,7]<stderr>:OMP: Info #209: KMP_AFFINITY: decoding x2APIC ids.
[1,7]<stderr>:OMP: Info #207: KMP_AFFINITY: Affinity capable, using global cpuid leaf 11 info
[1,7]<stderr>:OMP: Info #154: KMP_AFFINITY: Initial OS proc set respected: {0,1,2,3,4,5,6,7}
[1,7]<stderr>:OMP: Info #156: KMP_AFFINITY: 8 available OS procs
[1,7]<stderr>:OMP: Info #157: KMP_AFFINITY: Uniform topology
[1,7]<stderr>:OMP: Info #179: KMP_AFFINITY: 1 packages x 4 cores/pkg x 2 threads/core (4 total cores)
[1,7]<stderr>:OMP: Info #211: KMP_AFFINITY: OS proc to physical thread map:
[1,7]<stderr>:OMP: Info #171: KMP_AFFINITY: OS proc 0 maps to package 0 core 0 thread 0 
[1,7]<stderr>:OMP: Info #171: KMP_AFFINITY: OS proc 4 maps to package 0 core 0 thread 1 
[1,7]<stderr>:OMP: Info #171: KMP_AFFINITY: OS proc 1 maps to package 0 core 1 thread 0 
[1,7]<stderr>:OMP: Info #171: KMP_AFFINITY: OS proc 5 maps to package 0 core 1 thread 1 
[1,7]<stderr>:OMP: Info #171: KMP_AFFINITY: OS proc 2 maps to package 0 core 2 thread 0 
[1,7]<stderr>:OMP: Info #171: KMP_AFFINITY: OS proc 6 maps to package 0 core 2 thread 1 
[1,7]<stderr>:OMP: Info #171: KMP_AFFINITY: OS proc 3 maps to package 0 c

In [11]:
X, A, y = load_data(path=PATH,dataset=DATASET)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.

In [12]:
#more about pickle
#https://stackoverflow.com/questions/44144584/typeerror-cant-pickle-thread-lock-objects
#https://github.com/keras-team/keras/issues/8343
#https://github.com/keras-team/keras/issues/10528
#https://bugs.python.org/issue29168
#https://www.reddit.com/r/learnpython/comments/bl2vze/what_does_the_error_message_typeerror_cant_pickle/
#https://www.reddit.com/r/learnpython/comments/brbplc/cant_pickle_threadrlock_objects/
#https://www.bountysource.com/issues/66242430-typeerror-can-t-pickle-_thread-rlock-objects-while-saving-the-keras-model-using-model-save
#https://devrant.com/rants/1279739/typeerror-cant-pickle-thread-lock-objects-someone-please-guide-me-pick-dump-mode
#https://bugzilla.redhat.com/show_bug.cgi?id=1444983

In [13]:
# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))